In [1]:
import pandas as pd

In [2]:
name_doc = "../raw_data_spss/desc_columns.txt"
df = pd.read_csv(name_doc, sep="|")
df.columns


Index(['Variable                       ', 'Posición',
       'Etiqueta                                                                                                ',
       'Nivel de medición', 'Rol    ', 'Ancho de columna', 'Alineación',
       'Formato de impresión', 'Formato de grabación', 'Valores perdidos',
       'Unnamed: 10'],
      dtype='object')

In [3]:
columns = [column.strip() for column in df.columns]
columns

['Variable',
 'Posición',
 'Etiqueta',
 'Nivel de medición',
 'Rol',
 'Ancho de columna',
 'Alineación',
 'Formato de impresión',
 'Formato de grabación',
 'Valores perdidos',
 'Unnamed: 10']

In [4]:
df.columns = columns
df = df[['Variable', 'Etiqueta']]
df

,Variable,Etiqueta
0,N°_diada,<ninguno> ...
1,BienestarV1,<ninguno> ...
2,BienestarV2,<ninguno> ...
3,BienestarAv,<ninguno> ...
4,Fecha_entrevista,Fecha Primera entrevista ...
...,...,...
452,ads_a_i_pv_2_r,recodificaion afecto en 0 1 ...
453,ads_p_i_pv_r,recodificaion sosten en 0 1 ...
454,countinsecurei_2020,<ninguno> ...
455,countsecurei_2020,<ninguno> ...


In [5]:
matrix_data = []

for index in df.index:
    variable = df['Variable'][index].strip()
    etiqueta = df['Etiqueta'][index].strip()

    row = [variable, etiqueta]
    matrix_data.append(row)

df_data = pd.DataFrame(matrix_data, columns=['variable', 'description'])
df_data
    

,variable,description
0,N°_diada,<ninguno>
1,BienestarV1,<ninguno>
2,BienestarV2,<ninguno>
3,BienestarAv,<ninguno>
4,Fecha_entrevista,Fecha Primera entrevista
...,...,...
452,ads_a_i_pv_2_r,recodificaion afecto en 0 1
453,ads_p_i_pv_r,recodificaion sosten en 0 1
454,countinsecurei_2020,<ninguno>
455,countsecurei_2020,<ninguno>


In [6]:
df_data.to_csv("../raw_data_spss/description_categories.csv", index=False, encoding="utf-8")

In [7]:
name_input = "../raw_data_spss/valores_variables.txt"

open_doc = open(name_input, 'r', encoding="utf-8")

all_data = []

line = open_doc.readline()
while line:
    line = line.replace("\n", "")
    all_data.append(line)
    line = open_doc.readline()

open_doc.close()

all_data

['|Observador_1                  |1,00 |Mario                                                                                                 |',
 '|                              |-----|------------------------------------------------------------------------------------------------------|',
 '|                              |2,00 |Monserrat                                                                                             |',
 '|                              |-----|------------------------------------------------------------------------------------------------------|',
 '|                              |3,00 |Sebastian                                                                                             |',
 '|------------------------------|-----|------------------------------------------------------------------------------------------------------|',
 '|Género_bebé                   |0    |Femenino                                                                            

In [8]:
def process_data_element(value_to_process):
    matrix_data = []

    for value in value_to_process:

        if "---" not in value:
            value = value[1:-1]
            values = value.split("|")
            formated_value = []
            for element in values:
                element = element.strip()
                if len(element) >0:
                    formated_value.append(element)
            
            matrix_data.append(formated_value)

    length_values = [len(element) for element in matrix_data]
    max_lenght = max(length_values)

    key_value_index = -1
    key_value = ""

    for i in range(len(matrix_data)):
        if len(matrix_data[i]) == max_lenght:
            key_value_index = i
            key_value = matrix_data[i][0]

    new_row = [matrix_data[key_value_index][1], matrix_data[key_value_index][2]]

    dict_data = {key_value : [
        {"nominal_value" : new_row[0],
        "description" : new_row[1]}
    ]}

    for i in range(len(matrix_data)):
        if i != key_value_index:
            dic_element = {"nominal_value" : matrix_data[i][0], "description" : matrix_data[i][1]}
            dict_data[key_value].append(dic_element)

    return dict_data

In [9]:
index_data = []

for i in range(len(all_data)):
    if "|------------------------------|" in all_data[i] :
        index_data.append(i)

first_value = []

for i in range(index_data[0]):
    first_value.append(all_data[i])

first_value

['|Observador_1                  |1,00 |Mario                                                                                                 |',
 '|                              |-----|------------------------------------------------------------------------------------------------------|',
 '|                              |2,00 |Monserrat                                                                                             |',
 '|                              |-----|------------------------------------------------------------------------------------------------------|',
 '|                              |3,00 |Sebastian                                                                                             |']

In [10]:
array_data_to_process = [first_value]

for i in range(len(index_data)-1):
    start = index_data[i]
    stop = index_data[i+1]
    
    data_values = [all_data[k] for k in range(start, stop)]
    array_data_to_process.append(data_values)

In [11]:
dict_data_values_with_info = []

for element in array_data_to_process:
    dict_data = process_data_element(element)
    dict_data_values_with_info.append(dict_data)


In [12]:
import json 

with open("../raw_data_spss/desc_columns.json", 'w', encoding="utf-8") as data_export:
    json.dump(dict_data_values_with_info, data_export, ensure_ascii=False)